In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from Libreria.cargar_configbd import cargar_configbd
from Libreria.cargar_mes_a_procesar import cargar_mes_a_procesar

In [2]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
# Crear un cursor para ejecutar consultas
cur = conn.cursor()

In [3]:
mes, año = cargar_mes_a_procesar.leer_csv_en_lista('mes_a_procesar.csv')
mes = 'abr'
año = 2024
print(f"{mes}{año}")

abr2024


In [4]:
def detectar_segmento_caminata(viaje, umbral_velocidad, umbral_aceleracion, tiempo_espera):
    def tiempo_segmento(t1, t2):
        return (t2 - t1).total_seconds()
    
    def es_segmento_caminata(segmento_actual_caminata, tiempo_espera):
        if (len(segmento_actual_caminata) > 1 and pd.concat(segmento_actual_caminata)['speed'].quantile(0.25) < 1.3 
            and tiempo_segmento(segmento_actual_caminata[0]['recorded_at'], segmento_actual_caminata[-1]['recorded_at']) > tiempo_espera):
            return True
        return False
    
    def asignar_segid(lista_dfs_segmentos):
        def get_first_datetime(df, datetime_column):
            return df[datetime_column].min()  # Suponiendo que queremos la más temprana en caso de no estar ordenado.

        def sort_dataframes_by_first_datetime(dfs, datetime_column):
            # Ordenar la lista de DataFrames basándonos en la primera fecha y hora
            sorted_dfs = sorted(dfs, key=lambda df: get_first_datetime(df, datetime_column))
            return sorted_dfs

        # Ordenar la lista de segmentos por la hora inicial de cada segmento
        sorted_dataframes = sort_dataframes_by_first_datetime(lista_dfs_segmentos, 'recorded_at')
        # Asignar segid
        for idx, segment in enumerate(sorted_dataframes):
            segment['segid'] = idx +1
        # Unir los segmentos en uun solo df
        df_concatenado = pd.concat(sorted_dataframes, ignore_index=True)

        return df_concatenado[df_concatenado['etiqueta'] == 'caminata'], df_concatenado[df_concatenado['etiqueta'] != 'caminata']
    
    def corregir_outliers_caminata(df_viaje, umbral_velocidad, umbral_aceleracion):
        df = df_viaje.reset_index()
        df_corregido = []
        previous_speed = 0
        for idx, row in df.iterrows():

            med_speed = row['speed']
            aceleration = row['aceleration']
            flag = False
            if row['speed'] > umbral_velocidad:
                if idx > 3:
                    med_speed = df.iloc[idx - 4: idx-1]['speed'].mean()
                    flag = True
                else: 
                    med_speed = df.iloc[idx + 1:idx + 4]['speed'].mean()
                    flag = True
            row['speed'] = med_speed

            if flag:
                if idx > 0:
                    # Calcular la aceleración
                    time_diff = (df.at[idx, 'recorded_at'] - df.at[idx-1, 'recorded_at']).total_seconds()
                    speed_diff = med_speed - df.at[idx-1, 'speed']
                    aceleration = speed_diff / time_diff if time_diff != 0 else 0
                    row['aceleration'] = aceleration
                else:
                    row['aceleration'] = 0

            if row['aceleration'] > umbral_aceleracion:
                row['aceleration'] = df['aceleration'].mean()

            df_corregido.append(row)
        
        return pd.DataFrame(df_corregido, columns=['tripid', 'uid', 'latitude', 'longitude', 'distance', 'speed', 'aceleration', 'bearing', 'recorded_at', 'segid', 'etiqueta'])


    def convertir_listas_a_df(segmentos_caminata, segmentos_no_caminata):

        # Concatena todas las filas segmentos caminata en un solo DataFrame
        df_segmentos_caminata = []
        dfs_si = []
        if len(segmentos_caminata) > 0:
            for segmento_caminata in segmentos_caminata:
                if len(segmento_caminata) > 1:
                    segmento_caminata = pd.DataFrame(segmento_caminata, columns=viaje.keys())
                    segmento_caminata['etiqueta'] = 'caminata'
                    dfs_si.append(segmento_caminata)
                #df_segmentos_caminata = pd.concat(asignar_segid(dfs_si, id_inicio = 0), ignore_index=True)
                #df_segmentos_caminata = corregir_outliers_caminata(df_segmentos_caminata, umbral_velocidad=5, umbral_aceleracion=3)
                #df_segmentos_caminata['etiqueta'] = 'caminata'

        # Concatena todas las filas en un solo DataFrame
        df_segmentos_no_caminata = []
        dfs_no = []
        if len(segmentos_no_caminata) > 0:
            for segmento_no_caminata in segmentos_no_caminata:
                if len(segmento_no_caminata) > 1:
                    segmento_no_caminata = pd.DataFrame(segmento_no_caminata, columns=viaje.keys())
                    segmento_no_caminata['etiqueta'] = ''
                    dfs_no.append(segmento_no_caminata)
                #df_segmentos_no_caminata = pd.concat(asignar_segid(dfs_no, id_inicio = len(dfs_si)), ignore_index=True)
        
        df_segmentos_caminata, df_segmentos_no_caminata = asignar_segid(dfs_si + dfs_no)

        if len(df_segmentos_caminata) > 0:
            df_segmentos_caminata = corregir_outliers_caminata(df_segmentos_caminata, umbral_velocidad=5, umbral_aceleracion=3)

        return df_segmentos_caminata, df_segmentos_no_caminata

    segmentos_caminata = []
    segmentos_no_caminata = []
    segmento_actual_caminata = []
    segmento_actual_no_caminata = []
    
    flags = [] # Para saber si antes de la caminata actual hubo secuencia caminata, no_caminata
    pts_cam_rechazada = 0
    for idx, fila in viaje.iterrows(): # Itera puntos del viaje
        
        velocidad = fila['speed']
        aceleracion = fila['aceleration']

        if velocidad < umbral_velocidad and aceleracion < umbral_aceleracion: # Si es punto de caminata
            
            if len(segmento_actual_no_caminata) > 0: # En caso de que estaba formando segmento que no era caminata hay que guardar
                segmentos_no_caminata.append(segmento_actual_no_caminata)
                segmento_actual_no_caminata = []

                # secuencia de segmentos
                if len(flags) == 2:
                    flags.pop(0)
                flags.append(False)

            # Secuencia caminata, no_caminata,caminata (para analizar si el segmento del medio le absorbemos como caminata)
            if len(flags) == 2 and flags[0] == True and flags[1] == False and len(segmentos_no_caminata[-1]) < 3:
                segmento_actual_caminata = segmentos_caminata.pop() # Recupero la caminata antes de no caminata 
                segmento_actual_caminata += segmentos_no_caminata.pop()
                flags.pop()

            # Si los primeros puntos del viaje son caminata rechazada pero son pocos
            if len(flags) == 1 and flags[0] == False and pts_cam_rechazada != 0 and len(segmentos_no_caminata[-1]) - pts_cam_rechazada < 3: 
               segmento_actual_caminata = segmentos_no_caminata.pop()
               flags = []

            segmento_actual_caminata.append(fila)
            pts_cam_rechazada = 0

        else: # Si no es punto de caminata
            # Si cumple para ser segmento de caminata
            ## Si el principio del viaje es un segmento corto caminata puede haber estado subiendose al carro o en casa, cuando len flags es 0 es el principio
            if es_segmento_caminata(segmento_actual_caminata, tiempo_espera): 
                segmentos_caminata.append(segmento_actual_caminata)
                segmento_actual_caminata = []

                # secuencia de segmentos
                if len(flags) == 2:
                    flags.pop(0)
                flags.append(True)
                
            # Segmentos clasificados como caminata rechazados (puntos en los que carros se paran)
            elif len(segmento_actual_caminata) > 0: 
                if len(segmentos_no_caminata) > 0: 
                    segmento_actual_no_caminata = segmentos_no_caminata.pop() # Recupero la no caminata antes de caminata rechazada

                pts_cam_rechazada = len(segmento_actual_caminata)
                segmento_actual_no_caminata += segmento_actual_caminata 
                segmento_actual_caminata = []
            
            segmento_actual_no_caminata.append(fila)

    # Si el ultimo segmento es caminata y el utlimo segmento no caminata tiene menos de 3 puuntos
    if len(segmento_actual_caminata) > 0 and len(segmentos_no_caminata) > 0 and len(segmentos_no_caminata[-1]) < 3:
        # Si el ultimo segmento no caminata esta despuues de otra caminata es decir entre dos caminatas
        if len(flags) == 2 and flags[0] == True and flags[1] == False:
            if len(segmentos_caminata) > 0:
                segmento_actual_caminata += segmentos_caminata.pop() # Recupero la caminata antes de no caminata 
            segmento_actual_caminata += segmentos_no_caminata.pop()

    # Si cumple para ser segmento de caminata
    if es_segmento_caminata(segmento_actual_caminata, tiempo_espera): 
        segmentos_caminata.append(segmento_actual_caminata)
    # Segmentos clasificados como caminata rechazados
    elif len(segmento_actual_caminata) > 0: # Si es caminata rechazada
        if len(segmentos_no_caminata) > 0:
            segmento_actual_no_caminata =  segmentos_no_caminata.pop()
        segmentos_no_caminata.append(segmento_actual_no_caminata + segmento_actual_caminata)
    else: # Si el ultimo es no caminata
        if len(segmento_actual_no_caminata) > 2:
            segmentos_no_caminata.append(segmento_actual_no_caminata)
        elif len(segmento_actual_no_caminata) > 0 and len(segmentos_caminata) > 0:
            segmento_actual_caminata =  segmentos_caminata.pop()
            segmentos_caminata.append(segmento_actual_no_caminata + segmento_actual_caminata)
        elif len(segmento_actual_no_caminata) > 0:
            segmentos_no_caminata.append(segmento_actual_no_caminata)

    return convertir_listas_a_df(segmentos_caminata, segmentos_no_caminata) #df_caminata, df_no_caminata

In [5]:
# Crear tabla para insertar datos corregidos
consulta_sql = f'''DROP TABLE IF EXISTS segmentos_viajes_{mes}{año};
                    CREATE TABLE IF NOT EXISTS segmentos_viajes_{mes}{año}
                        (
                            tripid integer,
                            segid integer,
                            uid character varying(50),
                            latitude double precision,
                            longitude double precision,
                            distance double precision,
                            speed double precision,
                            aceleration double precision,
                            bearing double precision,
                            recorded_at timestamp without time zone
                        );

                    DROP TABLE IF EXISTS reglas_segmentos_caminata_{mes}{año};
                    CREATE TABLE IF NOT EXISTS reglas_segmentos_caminata_{mes}{año}
                        (
                            tripid integer,
                            segid integer,
                            uid character varying(50)
                        );
                    '''

cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()

In [6]:
cur.execute(f"SELECT DISTINCT tripid, uid FROM corregidos_viajes_{mes}{año};")
# Obtener los resultados de la consulta
rows = cur.fetchall()
id_viajes = rows.copy()

In [7]:
len(id_viajes)

33044

In [8]:
# ---- Segmentar viajes
    
print(f"n_viajes: {len(id_viajes)}")
prog = 0
for id_viaje in id_viajes:
    prog += 1
    print(f"Progreso: {prog}", end="\r")

    # Obtener los puntos del viaje
    cur.execute(f"SELECT * FROM corregidos_viajes_{mes}{año} where tripid='{id_viaje[0]}' and uid='{id_viaje[1]}' order by recorded_at")
    #print(f"SELECT * FROM corregidos_viajes_{mes}{año} where tripid='{id_viaje[0]}' and uid='{id_viaje[1]}' order by recorded_at")
    rows = cur.fetchall()
    puntos_viaje = rows.copy()
    df = pd.DataFrame(puntos_viaje, columns=['tripid', 'uid', 'latitude', 'longitude', 'distance', 'speed', 'aceleration', 'bearing', 'recorded_at'])
    
    # Obtener segmentos viaje
    #viaje, umbral_velocidad, umbral_aceleracion, tiempo_espera
    segmentos_caminata, segmentos_no_caminata = detectar_segmento_caminata(viaje=df, umbral_velocidad=2.5, umbral_aceleracion=0.2, tiempo_espera = 120)

    ###### insertar ids segmentos_camiunara en segmentos_caminata_sept2023
    if len(segmentos_caminata) > 0:
      for segid in segmentos_caminata['segid'].unique():
        consulta_sql = """
          INSERT INTO {} VALUES (
              %s, %s, %s
          )
          """.format(f'reglas_segmentos_caminata_{mes}{año}')

        # Ejecuta la consulta SQL con los valores de la fila actual
        cur.execute(consulta_sql, (id_viaje[0], int(segid), id_viaje[1]))

    # Verificar si encontro dos  tipos de segmentos o uno solo
    df = []
    if len(segmentos_caminata) > 0 and len(segmentos_no_caminata) > 0:
      df = pd.concat([segmentos_caminata, segmentos_no_caminata])
    elif len(segmentos_caminata) > 0:
      df = segmentos_caminata
    elif len(segmentos_no_caminata) > 0:
      df = segmentos_no_caminata

    # Insertar en tabla de viajes con segmentos
    for idx, row in df[['tripid', 'segid', 'uid', 'latitude', 'longitude', 'distance', 'speed', 'aceleration', 'bearing', 'recorded_at']].iterrows():
      # Construye la consulta SQL con parámetros
      consulta_sql = """
      INSERT INTO {} VALUES (
          %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
      )
      """.format(f'segmentos_viajes_{mes}{año}')

      # Ejecuta la consulta SQL con los valores de la fila actual
      cur.execute(consulta_sql, tuple(row))
    conn.commit()

n_viajes: 33044
